# CrewAI: Core Concepts and Working Examples

This notebook provides a comprehensive introduction to CrewAI, covering its core concepts and demonstrating them with working code examples.

## What is CrewAI?

CrewAI is a framework for orchestrating role-playing, autonomous AI agents. It allows multiple agents with different roles to collaborate and solve complex tasks. The framework is designed to facilitate the creation of specialized AI teams that can work together to accomplish goals.

## Setting Up the Environment

Let's start by installing CrewAI and the necessary dependencies:

In [ ]:
!pip install crewai crewai-tools

Now let's import the required modules:

In [1]:
import os
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool, WebsiteSearchTool

/opt/anaconda3/envs/getting_started_llmops/lib/python3.11/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/opt/anaconda3/envs/getting_started_llmops/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/opt/anaconda3/envs/getting_started_llmops/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap t

## Core Concepts

### 1. Agents

Agents are the building blocks of CrewAI. Each agent has a specific role, goal, and backstory, which defines its behavior and expertise.

Let's create our first agent:

In [2]:
# Define our LLM - using Ollama with Gemma 3
llm = LLM(model="ollama/gemma3:4b")

# Create a researcher agent
researcher = Agent(
    role="Research Analyst",
    goal="Find accurate and relevant information on the given topic",
    backstory="You are an experienced researcher with expertise in data collection and analysis. You have a knack for finding the most relevant information quickly.",
    verbose=True,
    llm=llm
)

# Create a writer agent
writer = Agent(
    role="Content Writer",
    goal="Create engaging and informative content based on research",
    backstory="You are a skilled writer with years of experience in creating compelling content. You excel at transforming complex information into clear, engaging narratives.",
    verbose=True,
    llm=llm
)

print(f"Created agents: {researcher.role} and {writer.role}")

Created agents: Research Analyst and Content Writer


### 2. Tasks

Tasks represent the specific actions that agents need to perform. Each task has a description, expected output, and is assigned to a specific agent.

In [3]:
# Create a research task
research_task = Task(
    description="Research the topic of 'Artificial Intelligence in Healthcare'. Focus on recent applications, benefits, and challenges.",
    expected_output="A comprehensive summary of AI applications in healthcare, including recent developments, benefits, and challenges.",
    agent=researcher
)

# Create a writing task
writing_task = Task(
    description="Using the research provided, write an informative article about 'Artificial Intelligence in Healthcare'. The article should be engaging, clear, and suitable for a general audience.",
    expected_output="A well-structured article about AI in healthcare that explains key concepts, applications, benefits, and challenges in an engaging way.",
    agent=writer
)

print(f"Created tasks for {research_task.agent.role} and {writing_task.agent.role}")

Created tasks for Research Analyst and Content Writer


### 3. Crews

A Crew is a team of agents working together to complete a set of tasks. The crew orchestrates the collaboration between agents and manages the execution of tasks.

In [4]:
# Create a crew with our agents and tasks
ai_healthcare_crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, writing_task],
    verbose=True,
    process=Process.sequential,  # Tasks will be executed one after another
)

print(f"Created crew with {len(ai_healthcare_crew.agents)} agents and {len(ai_healthcare_crew.tasks)} tasks")

Created crew with 2 agents and 2 tasks


### 4. Process Types

CrewAI supports different process types for task execution:

1. **Sequential Process**: Tasks are executed one after another in the order they are defined.
2. **Hierarchical Process**: A manager agent coordinates the crew, delegating tasks and validating outcomes.

Let's see how to use both process types:

In [5]:
# Sequential process (already shown above)
sequential_crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, writing_task],
    process=Process.sequential,
    verbose=True,
    llm=llm
)

# Hierarchical process
hierarchical_crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, writing_task],
    process=Process.hierarchical,
    verbose=True,
    manager_llm=llm  # Manager LLM is required for hierarchical process
)

print("Created crews with different process types: Sequential and Hierarchical")

Created crews with different process types: Sequential and Hierarchical


### 5. Tools

Tools extend the capabilities of agents, allowing them to perform specific actions like web searches or API calls.

In [6]:
# Create a web search tool (requires API key)
# For this example, we'll just define it without actual API keys
search_tool = WebsiteSearchTool()

# Create a new agent with tools
researcher_with_tools = Agent(
    role="Advanced Research Analyst",
    goal="Find accurate and relevant information on the given topic using various online sources",
    backstory="You are an experienced researcher with expertise in data collection and analysis. You leverage various tools to find the most relevant information quickly.",
    verbose=True,
    llm=llm,
    tools=[search_tool]
)

# Create a new task for this agent
advanced_research_task = Task(
    description="Research the topic of 'Artificial Intelligence in Healthcare'. Use the search tool to find recent applications, benefits, and challenges.",
    expected_output="A comprehensive summary of AI applications in healthcare, including recent developments, benefits, and challenges.",
    agent=researcher_with_tools
)

print(f"Created an agent with {len(researcher_with_tools.tools)} tool(s)")

Created an agent with 1 tool(s)


## Working Example: AI Technology Report Crew

Now, let's put everything together in a working example to create a report on AI technology:

In [8]:
# Define our LLM
llm = LLM(model="ollama/gemma3:4b")

# Create agents
tech_researcher = Agent(
    role="Technology Researcher",
    goal="Research cutting-edge AI technologies and their applications",
    backstory="You are a technology analyst with expertise in AI and emerging technologies. You stay up-to-date with the latest developments in the field.",
    verbose=True,
    llm=llm
)

tech_writer = Agent(
    role="Technology Writer",
    goal="Create clear and engaging reports on complex technology topics",
    backstory="You are a skilled technology writer who excels at explaining complex concepts in simple terms. Your articles are both informative and engaging.",
    verbose=True,
    llm=llm
)

# Create tasks
research_task = Task(
    description="Research the latest developments in artificial intelligence. Focus on recent breakthroughs, practical applications, and future trends.",
    expected_output="A detailed summary of recent AI developments, applications, and trends.",
    agent=tech_researcher
)

report_task = Task(
    description="Using the research provided, create a comprehensive report on 'The State of AI Technology in 2024'. The report should cover key developments, applications across industries, and future outlook.",
    expected_output="A well-structured, informative report on AI technology that covers recent developments, current applications, and future trends.",
    agent=tech_writer
)

# Create the crew
tech_report_crew = Crew(
    agents=[tech_researcher, tech_writer],
    tasks=[research_task, report_task],
    verbose=True,
    process=Process.sequential
)

# Run the crew
print("Running the AI Technology Report Crew...")
result = tech_report_crew.kickoff()

print("\n----- Final Report -----\n")
print(result)

Running the AI Technology Report Crew...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 79c7fcc1-7cf7-4bb5-868f-f021e5b16a06                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 5552636b-2eca-4ed6-b797-e7ca2000865e
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 5552636b-2eca-4ed6-b797-e7ca2000865e
       Status: Executing Task...
    └── 🤖 Agent: Technology Researcher
            Status: In Progress

# Agent: Technology Researcher
## Task: Research the latest developments in artificial intelligence. Focus on recent breakthroughs, practical applications, and future trends.


...

🚀 Crew: crew
├── 📋 Task: 5552636b-2eca-4ed6-b797-e7ca2000865e
│      Assigned to: Technology Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Technology Researcher
│           Status: ✅ Completed
└── 📋 Task: 4d364013-5e4a-4cf1-ba9e-3b79d05169fb
       Status: Executing Task...
    └── 🤖 Agent: Technology Writer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 5552636b-2eca-4ed6-b797-e7ca2000865e
│      Assigned to: Technology Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Technology Researcher
│           Status: ✅ Completed
└── 📋 Task: 4d364013-5e4a-4cf1-ba9e-3b79d05169fb
       Assigned to: Technology Writer
       Status: ✅ Completed
    └── 🤖 Agent: Technology Writer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4d364013-5e4a-4cf1-ba9e-3b79d05169fb                                                                     │
│  Agent: Technology Writer                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 79c7fcc1-7cf7-4bb5-868f-f021e5b16a06                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


----- Final Report -----

## The State of AI Technology in 2024: A Rapidly Evolving Landscape

Artificial Intelligence is experiencing a period of unprecedented growth and transformation in 2024, driven by breakthroughs in generative AI and a broadening range of applications across industries. This report outlines the key developments, current applications, and future outlook shaping this dynamic field.

**Key Developments & Trends:**

* **Dominance of Generative AI Models:** Models like GPT-4, Gemini, and Claude 3 remain at the forefront, showcasing remarkable abilities in generating text, images, code, and even video. Architectural innovations like Mixture of Experts (MoE) and Reinforcement Learning from Human Feedback (RLHF) are continually refining these models. Retrieval-Augmented Generation (RAG) is increasingly utilized to enhance accuracy and knowledge integration, addressing limitations in the models’ inherent knowledge.
* **Multimodal AI Ascendancy:** AI systems are no longe

## Memory and State Management in CrewAI

CrewAI allows agents to maintain memory across tasks, which is useful for context preservation and more complex interactions:

In [9]:
# Create agents with memory enabled
researcher_with_memory = Agent(
    role="Research Specialist",
    goal="Conduct in-depth research on specified topics",
    backstory="You are a meticulous researcher with a strong analytical mind. You excel at finding connections between different pieces of information.",
    verbose=True,
    llm=llm
)

writer_with_memory = Agent(
    role="Content Specialist",
    goal="Create engaging content based on research findings",
    backstory="You are a versatile writer who can adapt to different tones and styles. You excel at turning research into compelling narratives.",
    verbose=True,
    llm=llm
)

# Create tasks for a two-part project
initial_research_task = Task(
    description="Research the history and evolution of artificial intelligence from its inception to the present day.",
    expected_output="A chronological summary of AI development, highlighting key milestones and breakthroughs.",
    agent=researcher_with_memory
)

follow_up_research_task = Task(
    description="Building on your previous research, investigate the potential future developments in AI over the next decade.",
    expected_output="A summary of projected AI developments, based on current trends and expert predictions.",
    agent=researcher_with_memory
)

writing_task = Task(
    description="Using the research on AI history and future projections, write a comprehensive article titled 'AI: Past, Present, and Future'.",
    expected_output="A well-structured article that covers the evolution of AI and its potential future developments.",
    agent=writer_with_memory
)

# Create a crew with memory enabled
memory_crew = Crew(
    agents=[researcher_with_memory, writer_with_memory],
    tasks=[initial_research_task, follow_up_research_task, writing_task],
    verbose=True,
    process=Process.sequential,
    memory=True  # Enable memory for the crew
)

print("Created a crew with memory enabled")

Created a crew with memory enabled


## Example of Custom Tool Integration

Here's how to create and use a custom tool with CrewAI:

In [10]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type

# Define the input schema for our tool
class SummarizeTextInput(BaseModel):
    """Input schema for SummarizeTextTool."""
    text: str = Field(..., description="The text to be summarized.")
    max_length: int = Field(100, description="Maximum length of the summary in words.")

# Create a custom summarization tool
class SummarizeTextTool(BaseTool):
    name: str = "Summarize Text"
    description: str = "Summarizes a given text to a specified maximum length."
    args_schema: Type[BaseModel] = SummarizeTextInput

    def _run(self, text: str, max_length: int = 100) -> str:
        """Summarize the input text."""
        # In a real implementation, you might use an NLP library or API
        # For this example, we'll use a simple approach
        words = text.split()
        if len(words) <= max_length:
            return text
        
        return ' '.join(words[:max_length]) + '...'

# Create an agent with the custom tool
editor_agent = Agent(
    role="Content Editor",
    goal="Edit and refine content to improve clarity and conciseness",
    backstory="You are an experienced editor with a keen eye for detail. You excel at refining text to make it clearer and more engaging.",
    verbose=True,
    llm=llm,
    tools=[SummarizeTextTool()]
)

# Create a task for the editor agent
editing_task = Task(
    description="Review and edit the following text to make it more concise while preserving the key information:\n\n'Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. The term may also be applied to any machine that exhibits traits associated with a human mind such as learning and problem-solving. The ideal characteristic of artificial intelligence is its ability to rationalize and take actions that have the best chance of achieving a specific goal. AI has various applications across industries, including healthcare, finance, education, transportation, and entertainment. Recent advances in machine learning, particularly deep learning, have led to significant improvements in AI capabilities. However, the development of AI also raises ethical concerns and challenges related to privacy, security, and the potential impact on employment.'",
    expected_output="A concise, edited version of the text that preserves the key information.",
    agent=editor_agent
)

# Create a crew with the editor agent
editing_crew = Crew(
    agents=[editor_agent],
    tasks=[editing_task],
    verbose=True,
    process=Process.sequential
)

print("Created a crew with a custom tool")

Created a crew with a custom tool


## Conclusion

In this notebook, we've explored the core concepts of CrewAI:

1. **Agents**: The autonomous entities with specific roles, goals, and backstories.
2. **Tasks**: The specific actions assigned to agents.
3. **Crews**: Teams of agents working together to complete tasks.
4. **Processes**: Different ways to orchestrate task execution (Sequential and Hierarchical).
5. **Tools**: Extending agent capabilities with specific functionalities.
6. **Memory**: Maintaining context across tasks.

CrewAI provides a powerful framework for orchestrating AI agents to collaborate on complex tasks. By combining agents with different roles and expertise, you can create AI teams capable of tackling sophisticated problems.

In the next notebook, we'll explore how to build a book writer system using CrewAI's Crew and Flow functionalities.